In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

### 멜론  장르(OST, 국내영화, 국내드라마, 국내뮤지컬)

In [3]:
driver = webdriver.Chrome('C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe')

### 1. songId 만 가져와 파일로 저장하기

In [4]:
gn_code = [2, 4, 7]
songIds = []

base_url1 = 'https://www.melon.com/genre/song_list.htm?gnrCode=GN1500&dtlGnrCode=GN150'
base_url2 = '#params%5BgnrCode%5D=GN1500&params%5BdtlGnrCode%5D=GN150'
base_url3 = '&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex='

# base_url = 'https://www.melon.com/genre/song_list.htm?params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&params%5BdtlGnrCode%5D=params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&gnrCode=GN0'
# genre_url = '#params%5BgnrCode%5D=GN0'
# index_url = '&startIndex='
for i in gn_code:

    for k in range(1, 452, 50):

        page_url = f'{base_url1}{i}{base_url2}{i}{base_url3}{k}'
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('tbody > tr')
        for tr in trs:
            try:
                if tr.select_one('.bullet_icons.age_19'):
                    continue
                songId = tr.select_one('.btn.button_icons.type03.song_info')['href'].split("'")[-2]
                songIds.append(songId)
            except:
                print(page_url)



In [9]:
songid_df = pd.DataFrame(songIds, columns=['songId'])
songid_df.to_csv('data/songids.csv', index=False)

In [10]:
songid_df.shape

(1495, 1)

In [11]:
songid_df.nunique()

songId    1495
dtype: int64

##### 2. 파일을 읽어서 songId의 내용들 가져오기

In [ ]:
df = pd.read_csv('data/songids.csv')

In [17]:
infos = []
for songId in tqdm(songIds[0:1]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        print(song_url)

        driver.get(song_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        info = soup.select_one('#conts')

        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text()
        composer = ul[1].select_one('.artist_name').get_text()
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('.lyric')).split('\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip()
        try:
            buttons = info.select('.meta > button')
            plylstSeq = ' '.join([button['data-djcol-no'] for button in buttons])

        except:
            plylstSeq = np.nan

        infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                    'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                    'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue



  0%|          | 0/1 [00:00<?, ?it/s]

https://www.melon.com/song/detail.htm?songId=418168


100%|██████████| 1/1 [00:03<00:00,  3.95s/it]

https://cdnimg.melon.co.kr/cm/album/images/000/28/985/28985_500.jpg/melon/resize/282/quality/80/optimize


In [ ]:
df = pd.DataFrame(infos)
df.to_csv('data/melon_song_ost1.csv', encoding='utf-8', index=False)

In [16]:
df

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201,https://cdnimg.melon.co.kr/cm/album/images/000...,333,138152,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859
1,35609035,인생은 뷰티풀,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,원태연,20220907,https://cdnimg.melon.co.kr/cm2/album/images/11...,197,17307,하나만 알고 둘은\n잘 모르는 사람입니다\n하루는 비틀거리고\n하루는 뒤뚱거리고\n...,457536944 474137187
2,1046278,서쪽 하늘,이승철,발라드 국내영화,청연 OST,이승철,윤명선,20051220,https://cdnimg.melon.co.kr/cm/album/images/003...,84,48082,서쪽하늘로 노을은 지고 \n이젠 슬픔이 돼버린 그대를 \n다시 부를수 없을 것 ...,NaN
3,35609034,슬픈등,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,원태연,20220907,https://cdnimg.melon.co.kr/cm2/album/images/11...,144,15371,누군가를 다시 만난다면\n여전히 나는 너를\n누군가를 다시 사랑하면\n영원히 나는 ...,457536944 515278519
4,35728845,고백 (영화 '동감' X 츄 (이달의 소녀)),츄(Chuu),록/메탈 국내영화,고백 (영화 '동감' X 츄 (이달의 소녀)),강현민,강현민,20221016,https://cdnimg.melon.co.kr/cm2/album/images/11...,80,16882,말해야 하는데 네 앞에 서면\n아무 말 못 하는 내가 미워져\n용기를 내야 해 후회...,484732185 513801514
5,1046282,말리꽃,이승철,발라드 국내영화,청연 OST,이근상,박준배,20051220,https://cdnimg.melon.co.kr/cm/album/images/003...,55,55059,얼마나 더 견뎌야 하는지 \n짙은 어둠을 헤매고 있어 \n내가바란 꿈이라는 것은 \...,403014132 445029956
6,1525532,별,유미,발라드 국내영화,미녀는 괴로워 OST,이재학,이재학,20061211,https://cdnimg.melon.co.kr/cm/album/images/003...,82,44917,바람결이 창을 흔들고\n내 키만한 작은 나의 방 위로\n아름답게 별빛들은\n가득 채...,442706386 403014132
7,35837631,편지 (영화 '동감' X 윤하 (YOUNHA)),윤하 (YOUNHA),발라드 국내영화,편지 (영화 '동감' X 윤하 (YOUNHA)),허승경,김광진,20221116,https://cdnimg.melon.co.kr/cm2/album/images/11...,67,10269,여기까지가 끝인가 보오\n이제 나는 돌아서겠소\n억지 노력으로 인연을 거슬러\n괴롭...,496963017 456209348
8,7877288,꽃이 피고 지듯이,조승우,발라드 국내영화,사도 OST - 꽃이 피고 지듯이,방준석,방준석,20151007,https://cdnimg.melon.co.kr/cm/album/images/026...,162,36088,나 이제 가려합니다\n아픔은 남겨두고서\n당신과의 못다한 말들\n구름에 띄워놓고 가...,460637410 421338401
